#Segundo intento#



*   Solo con los datos de Madrid de Idealista2018
*   Madrid_Sale.csv y Madrid_Polygons.rda bajados de https://github.com/paezha/idealista18/tree/e2cfe3f621e1c37f226ce956f61b7306c42d5721/data


*   archivo Madrid_Polygons.rda transformado en csv con Notebook en R Script R rda a csv.ipyn https://colab.research.google.com/drive/1I06zwKUreseL1p87dHq0TzMheUHiEmVc


*   Exploración de datos (EDA) copiado de https://github.com/enricobonii/Idealista-Project/blob/main/IdealistaProject%20v4.ipynb
*Agregar la Columna neighbourhood a df_Madrid: de la latitud y longitud de cada casa se busca el barrio al que corresponde en Madrid_Polygons.csv


* a Madrid_Polygons se le añade la columna mean_UNITPRICE para utilizarla posteriormente para calcular precios de casas en 2024.

* Se intenta bajar la geometria de la M30, para calcular la distancia de los pisos a esta, y para definir si están dentro de la M30 o fuera. Como no funciona porque los datos de la M30 incluyen las carreteras radiales, y no se cierra, en las próximas versiones se desechará la M30

* Se calcula el número de estaciones de Metro que hay en un radio de 500m alrededor de cada casa, y se guarda en la nueva columna metro_within_500m

* finalmente, se guarda el dataframe como gdf_Madrid_with_metro_count.csv en el Google drive asi como Madrid_Polygons_with_mean_unitprice.csv

* lo siguiente, Multivariate Analysis etc. copiado de https://github.com/enricobonii/Idealista-Project/blob/main/IdealistaProject%20v4.ipynb no se ha tocado de momento





In [ ]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

# Ruta al archivo CSV en Google Drive
file_path = '/content/drive/My Drive/Idealista_2024/datos_2018/'

In [ ]:
from getpass import getpass
import os

# Solicita el token personal de GitHub
token = getpass('GitHub Token: ')

# Define la URL del repositorio y la ruta de clonación
repo_url = "https://github.com/edouardor/Idealista_2024.git"
destino = "/content/drive/MyDrive/Idealista_2024"

# Clona el repositorio en la carpeta de Google Drive
os.system(f'git clone https://{token}@github.com/edouardor/Idealista_2024.git {destino}')


In [ ]:
%cd /content/drive/MyDrive/Idealista_2024

In [ ]:
!ls -a


In [ ]:
!git config --global user.email "edouardo@hotmail.es"
!git config --global user.name "edouardor"

In [ ]:
# Agregar los archivos modificados
!git add .

# Hacer un commit
!git commit -m "Actualización desde Google Colab"

# Hacer push de los cambios
!git push


**Para ver las diferencias con el notebook anterior:**

*descomentar las lineas siguientes*

In [ ]:
#!pip install nbdime
#!nbdiff Notebooks/"Idealista Madrid github.ipynb"


In [ ]:
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from math import radians, cos, sin, asin, sqrt
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math

pd.set_option('display.max_columns', None)

## Initializing Datasets

###Solo Madrid


In [ ]:

df= pd.read_csv(file_path+'gdf_Madrid_with_metro_count.csv')


In [ ]:
gdf = pd.read_csv(file_path+'Madrid_Polygons_with_mean_unitprice.csv')

*para poner luego los clusters definidos para cada barrio*

In [ ]:
df.head(3)

## Exploratory Data Analysis

In [ ]:
print(df.dtypes)

From here we can observe that the formatted variable types make sense with the expected and are homogeneous among the different datasets.

In [ ]:
df.describe(include='all')

In [ ]:
# prompt: drop observertions where df[df['ROOMNUMBER']>20]

df = df[df['ROOMNUMBER'] <= 20]

## Multivariate Analysis

**Voy a cambiar la LATITUD y LONGITUD por la distancia en km a Nuevos Ministerios (AZCA), medida como si fuera una cuadrícula**

mido la distancia a un punto con la misma latitud que AZCA, y a otro con la misma longitud


*The coordinates are converted from latitude and longitude to northing and easting in meters, and then rescaled and centered using the corresponding city’s Central Business District as a false origin. These transformations have no impact on the analysis, and rescaling and centering of the coordinates is necessary for the correct implementation of the interactive basis functions in decision trees (see Antonio Paez et al. 2019, 188–89). https://github.com/paezha/Machine-learning-for-market-segmentation*

In [ ]:
# Definir la función de Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radio de la Tierra en km

    # Convertir las coordenadas de grados a radianes
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)

    # Aplicar la fórmula de Haversine
    a = np.sin(delta_phi / 2.0) ** 2 + \
        np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return R * c


# Coordenadas del punto de referencia
,
ref_lat = 40.446195
ref_lon = -3.691269

# Calcular las distancias
# Crear nuevas columnas 'DISTANCIA_LATITUD' y 'DISTANCIA_LONGITUD'
df['DISTANCIA_LATITUD'] = haversine(df['LATITUDE'], ref_lon, ref_lat, ref_lon)
df['DISTANCIA_LONGITUD'] = haversine(ref_lat, df['LONGITUDE'], ref_lat, ref_lon)

# Mostrar el DataFrame resultante
print(df.head())


In [ ]:
# Correlation analysis
'''
#CHANGE THE COLUMN CADCONSTRUCTIONYEAR datatype from int to year
df_Barcelona['CADCONSTRUCTIONYEAR'] = pd.to_datetime(df_Barcelona['CADCONSTRUCTIONYEAR'], format='%Y')
df_Barcelona['CADCONSTRUCTIONYEAR'] = df_Barcelona['CADCONSTRUCTIONYEAR'].dt.year

#Add a column CadBuildingAge in barcelona dataset where age is current year - cadconstructionyear
barcelona['CADBUILDINGAGE'] = dt.datetime.now().year - barcelona['CADCONSTRUCTIONYEAR']
barcelona = barcelona.drop(columns=['CADCONSTRUCTIONYEAR'])
'''

As per the above table, we can see that the columns CONSTRUCTEDAREA, BATHNUMBER, ROOMNUMBER, HASPARKINGSPACE, DISTANCE_TO_DIAGONAL, CADASTRALQUALITYID, ISPARKINGSPACEINCLUDEDINPRICE, HASDOORMAN, HASLIFT, HASTERRACE are some of the columns that have a higher correlation with the PRICE of the asset listed.


In [ ]:
madrid = df.drop(columns=['ADOPERATIONID', 'UNITPRICE', 'LATITUDE', 'LONGITUDE','AMENITYID','ASSETID','ADTYPOLOGY','CITYNAME','ADOPERATION','ADTYPOLOGYID'])

# Encode the location names (label encoding)
madrid['neighbourhood'] = df['neighbourhood'].astype('category').cat.codes

plt.figure(figsize=(16, 11))
correlation_madrid = madrid.corr()

# Identify columns where absolute correlation with 'PRICE' is less than 0.07
low_corr_columns = correlation_madrid['PRICE'][abs(correlation_madrid['PRICE']) < 0.07].index

# Drop the identified columns from the DataFrame
madrid_filtered = madrid.drop(columns=low_corr_columns)

plt.figure(figsize=(16, 11))
correlation_madrid = madrid_filtered.corr()
sns.heatmap(correlation_madrid, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
print(madrid.dtypes)

In [ ]:
# print list of columns from correlation matrix in decreasing order of correlation with Price
correlation_madrid['PRICE'].sort_values(ascending=False)

As per the above table, in Madrid,, we can see that the columns CONSTRUCTEDAREA, BATHNUMBER, ROOMNUMBER, HASDOORMAN, HASLIFT, HASPARKINGSPACE, metro_within_500 m, FloorClean are some of the columns that have a higher correlation with the PRICE of the asset listed.

**Analysing the pricing of assets based on location**

In [ ]:
madrid_unit = df.copy()
# Encode the location names (label encoding)
madrid_unit['neighbourhood'] = df['neighbourhood'].astype('category').cat.codes

# Calculate average price by neighbourhood
average_prices = madrid_unit.groupby('neighbourhood')['UNITPRICE'].mean().reset_index()

#Map the neighbourhoods to their respective names
neighbourhood_map = dict(zip(madrid_unit['neighbourhood'], df['neighbourhood']))
average_prices['neighbourhood'] = average_prices['neighbourhood'].map(neighbourhood_map)

# Sort the neighbourhoods by average price in descending order
average_prices = average_prices.sort_values(by='UNITPRICE', ascending=False)

# Plot the average prices by neighbourhood
plt.figure(figsize=(15, 25))
sns.barplot(x='UNITPRICE', y='neighbourhood', data=average_prices, palette='viridis')

plt.title('Average Asset Prices/unit by neighbourhood')
plt.ylabel('Neighbourhood')
plt.xlabel('Average Price')
plt.tight_layout()

# Show the plot
plt.show()

As we see above, in Madrid, the neighbourhoods with the highest average asset unit prices (price per meter square) are: Recoletos, Castellana, Almagro, and Chueca. So the asset to be listed in/around these areas can have a high asking price (around 6500/m2). Places such as San Andres, Entrevias or San Cristobal have the lowest average asset prices. So the asset to be listed in/around these areas can have a low asking price (around 1300/m2).

In [ ]:
madrid_unit['neighbourhood'].value_counts()

In [ ]:
df_Madrid = df.copy()

**Importamos las Librerías Necesarias**

In [ ]:
from sklearn.cluster import KMeans
import seaborn as sns

**calculamos el precio medio del metro cuadrado por barrio:**

In [ ]:
# Agrupar por barrio y calcular el precio medio del m²
precio_medio_barrio = df_Madrid.groupby('neighbourhood')['UNITPRICE'].mean().reset_index()

# Renombrar la columna para mayor claridad
precio_medio_barrio.rename(columns={'UNITPRICE': 'Precio_medio_m2'}, inplace=True)

print(precio_medio_barrio.head())


**Determinamos el Número Óptimo de Grupos**

*Antes de realizar la agrupación, es útil determinar cuántos grupos (clusters) son apropiados. Una técnica común para esto es el método del codo.*

In [ ]:
# Lista para almacenar la inercia
inercia = []
K = range(1, 10)

for k in K:
    modelo = KMeans(n_clusters=k, random_state=42)
    modelo.fit(precio_medio_barrio[['Precio_medio_m2']])
    inercia.append(modelo.inertia_)

# Graficar el método del codo
plt.figure(figsize=(8, 5))
plt.plot(K, inercia, 'bo-')
plt.xlabel('Número de clusters (k)')
plt.ylabel('Inercia')
plt.title('Método del Codo para Determinar el Número Óptimo de Clusters')
plt.show()


*Buscamos un "codo" en la gráfica donde la inercia comienza a disminuir de manera menos pronunciada. Aquí k=6*

**Aplicar K-Means para Agrupar los Barrios**

In [ ]:
# Definir el número de clusters
k_optimo = 6

# Inicializar y ajustar el modelo
kmeans = KMeans(n_clusters=k_optimo, random_state=42)
precio_medio_barrio['Cluster'] = kmeans.fit_predict(precio_medio_barrio[['Precio_medio_m2']])

# Opcional: Añadir los centroides
centroides = kmeans.cluster_centers_


In [ ]:
# Ordenar los barrios por precio medio y cluster
precio_medio_barrio.sort_values('Precio_medio_m2', inplace=True)

print(precio_medio_barrio)

# Visualización con Seaborn
plt.figure(figsize=(10, 6))
sns.scatterplot(data=precio_medio_barrio, x='neighbourhood', y='Precio_medio_m2', hue='Cluster', palette='viridis')
plt.xticks(rotation=90)
plt.xlabel('Barrio')
plt.ylabel('Precio Medio m² (€)')
plt.title('Agrupación de Barrios por Precio Medio del Metro Cuadrado')
plt.legend(title='Cluster')
plt.tight_layout()
plt.show()


**Nos Aseguramos la Consistencia de los Nombres de Barrios entre los ficheros**

In [ ]:
# Convertir a mayúsculas y eliminar espacios adicionales para asegurar coincidencias
precio_medio_barrio['neighbourhood'] = precio_medio_barrio['neighbourhood'].str.upper().str.strip()
gdf['LOCATIONNAME'] = gdf['LOCATIONNAME'].str.upper().str.strip()

# Opcional: Verificar barrios que no coinciden
barrios_df = set(precio_medio_barrio['neighbourhood'])
barrios_gdf = set(gdf['LOCATIONNAME'])
barrios_faltantes = barrios_df - barrios_gdf
if barrios_faltantes:
    print("Barrios en precio_medio_barrio que no están en gdf:", barrios_faltantes)

barrios_faltantes_gdf = barrios_gdf - barrios_df
if barrios_faltantes_gdf:
    print("Barrios en gdf que no están en precio_medio_barrio:", barrios_faltantes_gdf)


In [ ]:
gdf.head()

In [ ]:
import geopandas as gpd
from shapely import wkt

# Asegúrate de que 'WKT_geometry' está en formato string
gdf['WKT_geometry'] = gdf['WKT_geometry'].astype(str)

# Convertir la columna WKT a objetos geométricos
gdf['geometry'] = gdf['WKT_geometry'].apply(wkt.loads)


# Asegurarse de que 'gdf' es un GeoDataFrame
# Si no lo es, convertirlo
gdf = gpd.GeoDataFrame(gdf, geometry='geometry')  # Ajusta según corresponda

# Fusionar los DataFrames
gdf = gdf.merge(precio_medio_barrio[['neighbourhood', 'Cluster']], how='left',
                left_on='LOCATIONNAME', right_on='neighbourhood')

# Opcional: Verificar si hay valores NaN en 'Cluster' después de la fusión
barrios_sin_cluster = gdf[gdf['Cluster'].isna()]['LOCATIONNAME'].unique()
if len(barrios_sin_cluster) > 0:
    print("Barrios sin asignar a un cluster:", barrios_sin_cluster)

# Eliminar la columna 'Barrio' si ya no es necesaria
gdf.drop(columns=['neighbourhood'], inplace=True)


In [ ]:
# Supongamos que tu GeoDataFrame se llama 'gdf' y tiene una columna 'Cluster'
# y una columna 'mean_UNITPRICE' que representa el precio medio por barrio.

# Calcular el precio medio por cluster
precio_medio_por_cluster = gdf.groupby('Cluster')['mean_UNITPRICE'].mean().reset_index()

print(precio_medio_por_cluster)


In [ ]:
# Ordenar los clusters por 'mean_UNITPRICE' de menor a mayor
precio_medio_por_cluster = precio_medio_por_cluster.sort_values('mean_UNITPRICE').reset_index(drop=True)

print(precio_medio_por_cluster)


In [ ]:
# Crear una lista de clusters ordenados por precio medio
clusters_ordenados = precio_medio_por_cluster['Cluster'].tolist()

# Crear un diccionario de mapeo: {cluster_original: cluster_nuevo}
mapeo_clusters = {cluster_original: nuevo_cluster for nuevo_cluster, cluster_original in enumerate(clusters_ordenados)}

print(mapeo_clusters)


In [ ]:
# Aplicar el mapeo para crear una nueva columna 'Cluster_Ordenado'
gdf['Cluster_Ordenado'] = gdf['Cluster'].map(mapeo_clusters)

# Opcional: Si deseas reemplazar la columna 'Cluster' original
# gdf_geo['Cluster'] = gdf_geo['Cluster'].map(mapeo_clusters)


In [ ]:
# Definir una paleta de colores
palette = sns.color_palette('coolwarm', k_optimo).as_hex()

# Asignar colores a cada cluster
gdf['color'] = gdf['Cluster_Ordenado'].apply(lambda x: palette[x])

# Plotear el mapa
fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(color=gdf['color'], edgecolor='black', ax=ax)

# Añadir una leyenda
for cluster in range(k_optimo):
    ax.scatter([], [], c=palette[cluster], label=f'Cluster {cluster}')
ax.legend(title='Cluster')

plt.title('Agrupación de Barrios por Precio Medio del Metro Cuadrado')
plt.show()


In [ ]:
gdf.head()

**Seleccionamos las Columnas Relevantes de gdf**


In [ ]:
# Seleccionar columnas necesarias
clusters_info = gdf[['LOCATIONNAME', 'Cluster_Ordenado']]


In [ ]:
clusters_info

In [ ]:
# Convertir a mayúsculas y eliminar espacios al inicio y al final
df_Madrid['neighbourhood'] = df_Madrid['neighbourhood'].str.upper().str.strip()
gdf['LOCATIONNAME'] = gdf['LOCATIONNAME'].str.upper().str.strip()


**Realizamos la Fusión (Merge) de df_Madrid con gdf**

In [ ]:
# Fusionar df_Madrid con clusters_info
df_Madrid = df_Madrid.merge(clusters_info, how='left', left_on='neighbourhood', right_on='LOCATIONNAME')

# Opcional: Eliminar la columna 'LOCATIONNAME' si ya no es necesaria
df_Madrid.drop(columns=['LOCATIONNAME'], inplace=True)


In [ ]:
df_Madrid.head()

**Verificamos la Asignación de Clusters**

In [ ]:
# Verificar si hay pisos sin asignar a un cluster
pisos_sin_cluster = df_Madrid[df_Madrid['Cluster_Ordenado'].isna()]
num_sin_cluster = pisos_sin_cluster.shape[0]

if num_sin_cluster > 0:
    print(f"Hay {num_sin_cluster} pisos sin asignar a un cluster:")
    print(pisos_sin_cluster[['neighbourhood']].drop_duplicates())
else:
    print("Todos los pisos han sido asignados a un cluster.")


In [ ]:
sns.boxplot(data=df_Madrid, x='Cluster_Ordenado', y='UNITPRICE')

plt.title('Distribución de Precios por Cluster de Barrio')
plt.xlabel('Cluster Ordenado')
plt.ylabel('Precio Medio m² (€)')
plt.show()


## ML Models

**Cambiamos el precio por su logaritmo neperiano, para tener una distrubución más normal**

In [ ]:
#df_Madrid['PRICE'] = df_Madrid['PRICE'].apply(lambda x: np.log(x) if x > 0 else 0)

In [ ]:
df_Madrid.columns

In [ ]:
columns_to_keep = [
    'PRICE', 'CONSTRUCTEDAREA', 'ROOMNUMBER', 'BATHNUMBER', 'HASTERRACE',
    'HASLIFT', 'HASAIRCONDITIONING', 'HASSWIMMINGPOOL', 'ISSTUDIO',
    'ISINTOPFLOOR', 'HASPARKINGSPACE', 'HASGARDEN', 'HASBALCONY',
    'DISTANCE_TO_METRO','metro_within_500','Cluster_Ordenado'
]


df_Madrid_abridged = df_Madrid[columns_to_keep]



"\ncolumns_to_keep = [\n    'PRICE', 'CONSTRUCTEDAREA', 'ROOMNUMBER', 'BATHNUMBER', 'HASTERRACE',\n    'HASLIFT', 'HASAIRCONDITIONING', 'HASSWIMMINGPOOL', 'ISSTUDIO',\n    'ISINTOPFLOOR', 'HASPARKINGSPACE', 'DISTANCE_TO_CITY_CENTER',\n    'DISTANCE_TO_METRO','NEIGHBOURHOOD_ENCODED','metro_within_500'\n]\n\ndf_Barcelona = df_Barcelona[columns_to_keep]\ndf_Madrid = df_Madrid[columns_to_keep]\ndf_Valencia = df_Valencia[columns_to_keep]\n"

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error
import statsmodels.api as sm

X = df_Madrid.drop(['PRICE'], axis=1)
y = df_Madrid['PRICE']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_sm = sm.add_constant(X_train)
X_test_sm = sm.add_constant(X_test)

model_sm = sm.OLS(y_train, X_train_sm).fit()

y_pred = model_sm.predict(X_test_sm)

mae = mean_absolute_error(y_test, y_pred)
mdae = median_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print(f'MAE: {mae}')
print(f'MDAE: {mdae}')
print(f'RMSE: {rmse}')
print(f'MAPE: {mape}%')

print(model_sm.summary())

MAE: 91511.84049400924
MDAE: 58244.5621462764
RMSE: 165772.85247170023
MAPE: 31.469272608989545%
                            OLS Regression Results                            
Dep. Variable:                  PRICE   R-squared:                       0.845
Model:                            OLS   Adj. R-squared:                  0.845
Method:                 Least Squares   F-statistic:                 1.177e+04
Date:                Sat, 26 Oct 2024   Prob (F-statistic):               0.00
Time:                        19:41:01   Log-Likelihood:            -8.1315e+05
No. Observations:               60590   AIC:                         1.626e+06
Df Residuals:                   60561   BIC:                         1.627e+06
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------

In [ ]:
np.exp(0.1849)

1.2030981242995742

In [ ]:
np.exp(12.49)

265667.28590869467

In [ ]:

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
#from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error
#import matplotlib.pyplot as plt

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestRegressor(random_state=42)

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

grid_search.fit(X_train, y_train)

best_rf = grid_search.best_estimator_

y_pred = best_rf.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mdae = median_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print(f'MAE: {mae}')
print(f'MDAE: {mdae}')
print(f'RMSE: {rmse}')
print(f'MAPE: {mape}%')

print(f'Best parameters found: {grid_search.best_params_}')

feature_importances = best_rf.feature_importances_
features = X.columns

importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print(importance_df)

plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Feature Importance in Random Forest Model')
plt.gca().invert_yaxis()
plt.show()

Fitting 5 folds for each of 162 candidates, totalling 810 fits


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb = XGBRegressor(random_state=42)

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.1, 0.5, 1],
    'reg_lambda': [0, 0.1, 0.5, 1]
}

grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

grid_search.fit(X_train, y_train)

best_xgb = grid_search.best_estimator_

y_pred = best_xgb.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mdae = median_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print(f'MAE: {mae}')
print(f'MDAE: {mdae}')
print(f'RMSE: {rmse}')
print(f'MAPE: {mape}%')

print(f'Best parameters found: {grid_search.best_params_}')

Fitting 5 folds for each of 6912 candidates, totalling 34560 fits


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


MAE: 0.11357728456543496
MDAE: 0.08306807880904721
RMSE: 0.15843951980475932
MAPE: 0.9022131472068953%
Best parameters found: {'colsample_bytree': 0.6, 'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 300, 'reg_alpha': 0.1, 'reg_lambda': 0, 'subsample': 1.0}
